# Generating Multiple Benchmark Flow Traffic Sets

In this example, we write a script which will generate multiple benchmark traffic sets in a loop and save them in .pickle format. We will assume we are generating traffic for a `TrafPy` fat tree topology, although of course you can generate traffic for any arbitrary topology defined outside of `TrafPy` (see documentation and other examples).

We will generate the rack distribution sensitivity benchmark data set for loads 0.1-0.5.

In [1]:
import trafpy.generator as tpg
from trafpy.benchmarker import BenchmarkImporter

import numpy as np
import time
import os
from collections import defaultdict # use for initialising arbitrary length nested dict
from sqlitedict import SqliteDict
import json
from pathlib import Path
import gzip
import pickle

## 1. Define Generation Configuration

If you were writing this in a script rather than a Jupyter Notebook, you may want to e.g. put this next cell in a `config.py` file and import the file into a separate script for conciseness.

In [2]:
# -------------------------------------------------------------------------
# general configuration
# -------------------------------------------------------------------------
# define benchmark version
BENCHMARK_VERSION = 'v001'

# define minimum number of demands to generate (may generate more to meet jensen_shannon_distance_threshold and/or min_last_demand_arrival_time)
MIN_NUM_DEMANDS = None
MAX_NUM_DEMANDS = 5000 

# define maximum allowed Jenson-Shannon distance for flow size and interarrival time distributions (lower value -> distributions must be more similar -> higher number of demands will be generated) (must be between 0 and 1)
JENSEN_SHANNON_DISTANCE_THRESHOLD = 0.3

# define minimum time of last demand's arrival (helps define minimum simulation time)
MIN_LAST_DEMAND_ARRIVAL_TIME = None 

# define network load fractions
LOADS = [round(load, 3) for load in np.arange(0.1, 0.4, 0.1).tolist()] # ensure no python floating point arithmetic errors

# define whether or not to TrafPy packer should auto correct invalid node distribution(s)
AUTO_NODE_DIST_CORRECTION = True

# slot size (if None, won't generate slots_dict database)
# SLOT_SIZE = None 
SLOT_SIZE = 1000.0 # 50.0 1000.0 10.0


# -------------------------------------------------------------------------
# benchmark-specific configuration
# -------------------------------------------------------------------------

BENCHMARKS = ['rack_sensitivity_0',
              'rack_sensitivity_02',
              'rack_sensitivity_04',
              'rack_sensitivity_06',
              'rack_sensitivity_08']

# define network topology for each benchmark
net = tpg.gen_fat_tree(k=4, 
                       L=2, 
                       n=8, 
                       num_channels=1, 
                       server_to_rack_channel_capacity=1250, # 1250
                       rack_to_edge_channel_capacity=1000, 
                       edge_to_agg_channel_capacity=1000, 
                       agg_to_core_channel_capacity=2000)
NETS = {benchmark: net for benchmark in BENCHMARKS}

# define network capacity for each benchmark
NETWORK_CAPACITIES = {benchmark: net.graph['max_nw_capacity'] for benchmark in BENCHMARKS}
NETWORK_EP_LINK_CAPACITIES = {benchmark: net.graph['ep_link_capacity'] for benchmark in BENCHMARKS}

# define network racks for each benchmark
RACKS_DICTS = {benchmark: net.graph['rack_to_ep_dict'] for benchmark in BENCHMARKS}

## 2. Write a Function to Generate the Benchmark Traffic

This function should use the above configuration variables to generate traffic for each of our benchmarks as required.

In [3]:
def gen_benchmark_demands(path_to_save=None, 
                          load_prev_dists=True,
                          overwrite=False):
    '''
    If slot size is not None, will also generate an sqlite database for the slots_dict
    dictionary. This is useful if later during simulations want to have pre-computed
    slots_dict rather than computing & storing them in memory.

    '''
    if path_to_save[-1] == '/' or path_to_save[-1] == '\\':
        path_to_save = path_to_save[:-1]

    # init benchmark importer
    importer = BenchmarkImporter(BENCHMARK_VERSION, load_prev_dists=load_prev_dists)

    # load distributions for each benchmark
    benchmark_dists = {benchmark: {} for benchmark in BENCHMARKS}

    nested_dict = lambda: defaultdict(nested_dict)
    benchmark_demands = nested_dict()

    # begin generating data for each benchmark
    num_loads = len(LOADS)
    start_loops = time.time()
    print('\n~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*')
    print('Benchmarks to Generate: {}'.format(BENCHMARKS))
    print('Loads to generate: {}'.format(LOADS))
    for benchmark in BENCHMARKS:
        print('~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*')
        print('Generating demands for benchmark \'{}\'...'.format(benchmark))
        
        # get racks and endpoints
        racks_dict = RACKS_DICTS[benchmark]
        if racks_dict is not None:
            eps_racks_list = [eps for eps in racks_dict.values()]
            eps = []
            for rack in eps_racks_list:
                for ep in rack:
                    eps.append(ep)
        else:
            eps = NETS[benchmark].graph['endpoints']

        start_benchmark = time.time()
        load_counter = 1
        benchmark_dists[benchmark] = importer.get_benchmark_dists(benchmark, eps, racks_dict=racks_dict)
        for load in LOADS:
            start_load = time.time()
            network_load_config = {'network_rate_capacity': NETWORK_CAPACITIES[benchmark], 
                                   'ep_link_capacity': NETWORK_EP_LINK_CAPACITIES[benchmark],
                                   'target_load_fraction': load,
                                   'disable_timeouts': True}
            print('Generating demand data for benchmark {} load {}...'.format(benchmark, load))
            if benchmark_dists[benchmark]['num_ops_dist'] is not None:
                # job-centric
                use_multiprocessing = True
            else:
                # flow-centric
                use_multiprocessing = False
            demand_data = tpg.create_demand_data(min_num_demands=MIN_NUM_DEMANDS,
                                             max_num_demands=MAX_NUM_DEMANDS, 
                                             eps=eps,
                                             node_dist=benchmark_dists[benchmark]['node_dist'],
                                             flow_size_dist=benchmark_dists[benchmark]['flow_size_dist'],
                                             interarrival_time_dist=benchmark_dists[benchmark]['interarrival_time_dist'],
                                             num_ops_dist=benchmark_dists[benchmark]['num_ops_dist'],
                                             c=3,
                                             jensen_shannon_distance_threshold=JENSEN_SHANNON_DISTANCE_THRESHOLD,
                                             network_load_config=network_load_config,
                                             min_last_demand_arrival_time=MIN_LAST_DEMAND_ARRIVAL_TIME,
                                             auto_node_dist_correction=AUTO_NODE_DIST_CORRECTION,
                                             use_multiprocessing=use_multiprocessing,
                                             print_data=False)

            file_path = path_to_save + '/benchmark_{}_load_{}'.format(benchmark, load)
            tpg.pickle_data(path_to_save=file_path, data=demand_data, overwrite=overwrite)

            # reset benchmark demands dict to save memory
            benchmark_demands = nested_dict()

            if SLOT_SIZE is not None:
                # generate slots dict and save as database
                print('Creating slots_dict database with slot_size {}...'.format(SLOT_SIZE))
                s = time.time()
                demand = tpg.Demand(demand_data, eps=eps)
                with SqliteDict(file_path+'_slotsize_{}_slots_dict.sqlite'.format(SLOT_SIZE)) as slots_dict:
                    for key, val in demand.get_slots_dict(slot_size=SLOT_SIZE, include_empty_slots=True, print_info=True).items():
                        if type(key) is not str:
                            slots_dict[json.dumps(key)] = val
                        else:
                            slots_dict[key] = val
                    slots_dict.commit()
                    slots_dict.close()
                e = time.time()
                print('Created slots_dict database in {} s'.format(e-s))
            else:
                pass

            end_load = time.time()
            print('Generated \'{}\' demands for load {} of {} in {} seconds.'.format(benchmark, load_counter, num_loads, end_load-start_load))
            load_counter += 1

        end_benchmark = time.time()
        print('Generated demands for benchmark \'{}\' in {} seconds.'.format(benchmark, end_benchmark-start_benchmark))

    end_loops = time.time()
    print('Generated all benchmarks in {} seconds.'.format(end_loops-start_loops))

    return benchmark_demands


## 3. Generate the Benchmark Traffic

We will generate each of our traffic sets 2x to enable us to run 2 repeat experiments for each set

In [4]:
for _set in range(2):
    path_to_save = '../data/generate_multiple_benchmark_traffic_sets/set_{}_benchmark_data'.format(_set)
    Path(path_to_save).mkdir(exist_ok=True, parents=True)
    benchmark_demands = gen_benchmark_demands(path_to_save=path_to_save,
                                              load_prev_dists=False,
                                              overwrite=False)

load_prev_dist=False. Will re-generate dists with given network params and override any previously saved distributions.

~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*
Benchmarks to Generate: ['rack_sensitivity_0', 'rack_sensitivity_02', 'rack_sensitivity_04', 'rack_sensitivity_06', 'rack_sensitivity_08']
Loads to generate: [0.1, 0.2, 0.3, 0.4]
~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*
Generating demands for benchmark 'rack_sensitivity_0'...
Set to save benchmark rack_sensitivity_0 distribution data to /home/cwfparsonson/Insync/zciccwf@ucl.ac.uk/OneDriveBiz/ipes_cdt/phd_project/projects/trafpy/trafpy/benchmarker/versions/benchmark_v001/data/rack_sensitivity_0/
Saved node_dist distribution data to /home/cwfparsonson/Insync/zciccwf@ucl.ac.uk/OneDriveBiz/ipes_cdt/phd_project/projects/trafpy/trafpy/benchmarker/versions/benchmark_v001/data/rack_sensitivity_0/
Saved flow_size_dist distribution data to /home/cwfparsonson/Insync/zciccwf@ucl.ac.uk/OneDriveBiz/ipes_cdt/phd_proje

Packed 4450 flows in 2.673919439315796 s.
Time to save data to ../data/generate_multiple_benchmark_traffic_sets/set_0_benchmark_data/benchmark_rack_sensitivity_0_load_0.1.pickle: 0.10365486145019531 s
Creating slots_dict database with slot_size 1000.0...
Generated slot dict in 0.0083 s with slot size 1000.0 and total session time 48398.71450000157 for 4450 demands.
Number of slots making up total session time: 50
Number of these slots in which no new demands arrived: 1
Fraction of the 50 total time slots from simulation start to finish in which no new demands arrive: 0.02
Average number of demands arriving per time slot: 89.0
Created slots_dict database in 0.04524993896484375 s
Generated 'rack_sensitivity_0' demands for load 1 of 4 in 3.3253774642944336 seconds.
Generating demand data for benchmark rack_sensitivity_0 load 0.2...


Packed 4450 flows in 2.7188119888305664 s.
Time to save data to ../data/generate_multiple_benchmark_traffic_sets/set_0_benchmark_data/benchmark_rack_sensitivity_0_load_0.2.pickle: 0.10370159149169922 s
Creating slots_dict database with slot_size 1000.0...
Generated slot dict in 0.0076 s with slot size 1000.0 and total session time 30356.248500001493 for 4450 demands.
Number of slots making up total session time: 32
Number of these slots in which no new demands arrived: 1
Fraction of the 32 total time slots from simulation start to finish in which no new demands arrive: 0.031
Average number of demands arriving per time slot: 139.062
Created slots_dict database in 0.039942026138305664 s
Generated 'rack_sensitivity_0' demands for load 2 of 4 in 3.3448901176452637 seconds.
Generating demand data for benchmark rack_sensitivity_0 load 0.3...


Packed 4450 flows in 2.6937170028686523 s.
Time to save data to ../data/generate_multiple_benchmark_traffic_sets/set_0_benchmark_data/benchmark_rack_sensitivity_0_load_0.3.pickle: 0.10617661476135254 s
Creating slots_dict database with slot_size 1000.0...
Generated slot dict in 0.0074 s with slot size 1000.0 and total session time 15423.03933333391 for 4450 demands.
Number of slots making up total session time: 17
Number of these slots in which no new demands arrived: 1
Fraction of the 17 total time slots from simulation start to finish in which no new demands arrive: 0.059
Average number of demands arriving per time slot: 261.765
Created slots_dict database in 0.03671145439147949 s
Generated 'rack_sensitivity_0' demands for load 3 of 4 in 3.3248164653778076 seconds.
Generating demand data for benchmark rack_sensitivity_0 load 0.4...


Packed 4450 flows in 2.7914135456085205 s.
Time to save data to ../data/generate_multiple_benchmark_traffic_sets/set_0_benchmark_data/benchmark_rack_sensitivity_0_load_0.4.pickle: 0.10705399513244629 s
Creating slots_dict database with slot_size 1000.0...
Generated slot dict in 0.0078 s with slot size 1000.0 and total session time 11075.655500000017 for 4450 demands.
Number of slots making up total session time: 13
Number of these slots in which no new demands arrived: 1
Fraction of the 13 total time slots from simulation start to finish in which no new demands arrive: 0.077
Average number of demands arriving per time slot: 342.308
Created slots_dict database in 0.03602194786071777 s
Generated 'rack_sensitivity_0' demands for load 4 of 4 in 3.425894021987915 seconds.
Generated demands for benchmark 'rack_sensitivity_0' in 27.411484479904175 seconds.
~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*
Generating demands for benchmark 'rack_sensitivity_02'...
Set to save benchmark rack_

...

Packed 4450 flows in 2.3463761806488037 s.
Time to save data to ../data/generate_multiple_benchmark_traffic_sets/set_1_benchmark_data/benchmark_rack_sensitivity_06_load_0.4.pickle: 0.10597920417785645 s
Creating slots_dict database with slot_size 1000.0...
Generated slot dict in 0.009 s with slot size 1000.0 and total session time 11088.157000000043 for 4450 demands.
Number of slots making up total session time: 13
Number of these slots in which no new demands arrived: 1
Fraction of the 13 total time slots from simulation start to finish in which no new demands arrive: 0.077
Average number of demands arriving per time slot: 342.308
Created slots_dict database in 0.038773298263549805 s
Generated 'rack_sensitivity_06' demands for load 4 of 4 in 3.0200839042663574 seconds.
Generated demands for benchmark 'rack_sensitivity_06' in 26.29509973526001 seconds.
~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*~*
Generating demands for benchmark 'rack_sensitivity_08'...
Set to save benchmark ra

Packed 4450 flows in 2.2089240550994873 s.
Time to save data to ../data/generate_multiple_benchmark_traffic_sets/set_1_benchmark_data/benchmark_rack_sensitivity_08_load_0.1.pickle: 0.10180234909057617 s
Creating slots_dict database with slot_size 1000.0...
Generated slot dict in 0.0085 s with slot size 1000.0 and total session time 43262.18499999797 for 4450 demands.
Number of slots making up total session time: 45
Number of these slots in which no new demands arrived: 2
Fraction of the 45 total time slots from simulation start to finish in which no new demands arrive: 0.044
Average number of demands arriving per time slot: 98.889
Created slots_dict database in 0.0448460578918457 s
Generated 'rack_sensitivity_08' demands for load 1 of 4 in 2.875056028366089 seconds.
Generating demand data for benchmark rack_sensitivity_08 load 0.2...


Packed 4450 flows in 2.1339192390441895 s.
Time to save data to ../data/generate_multiple_benchmark_traffic_sets/set_1_benchmark_data/benchmark_rack_sensitivity_08_load_0.2.pickle: 0.10098600387573242 s
Creating slots_dict database with slot_size 1000.0...
Generated slot dict in 0.0084 s with slot size 1000.0 and total session time 27561.315499999284 for 4450 demands.
Number of slots making up total session time: 29
Number of these slots in which no new demands arrived: 1
Fraction of the 29 total time slots from simulation start to finish in which no new demands arrive: 0.034
Average number of demands arriving per time slot: 153.448
Created slots_dict database in 0.04022669792175293 s
Generated 'rack_sensitivity_08' demands for load 2 of 4 in 2.8100943565368652 seconds.
Generating demand data for benchmark rack_sensitivity_08 load 0.3...


Packed 4450 flows in 2.1861681938171387 s.
Time to save data to ../data/generate_multiple_benchmark_traffic_sets/set_1_benchmark_data/benchmark_rack_sensitivity_08_load_0.3.pickle: 0.10334610939025879 s
Creating slots_dict database with slot_size 1000.0...
Generated slot dict in 0.055 s with slot size 1000.0 and total session time 14831.95683333334 for 4450 demands.
Number of slots making up total session time: 16
Number of these slots in which no new demands arrived: 1
Fraction of the 16 total time slots from simulation start to finish in which no new demands arrive: 0.062
Average number of demands arriving per time slot: 278.125
Created slots_dict database in 0.08539509773254395 s
Generated 'rack_sensitivity_08' demands for load 3 of 4 in 2.869316816329956 seconds.
Generating demand data for benchmark rack_sensitivity_08 load 0.4...


Packed 4450 flows in 2.146083116531372 s.
Time to save data to ../data/generate_multiple_benchmark_traffic_sets/set_1_benchmark_data/benchmark_rack_sensitivity_08_load_0.4.pickle: 0.1027841567993164 s
Creating slots_dict database with slot_size 1000.0...
Generated slot dict in 0.0094 s with slot size 1000.0 and total session time 10049.862749999986 for 4450 demands.
Number of slots making up total session time: 12
Number of these slots in which no new demands arrived: 1
Fraction of the 12 total time slots from simulation start to finish in which no new demands arrive: 0.083
Average number of demands arriving per time slot: 370.833
Created slots_dict database in 0.0386357307434082 s
Generated 'rack_sensitivity_08' demands for load 4 of 4 in 2.8203325271606445 seconds.
Generated demands for benchmark 'rack_sensitivity_08' in 27.1573326587677 seconds.
Generated all benchmarks in 136.3916130065918 seconds.
